In [7]:
import os
import torch
from torch.utils.data import DataLoader
import sys
sys.path.append('/net/cremi/sasifchaudhr/espaces/travail/Semantic-Segmentation-for-Autonomous-Driving')
from src.dataset import ProjectDatasets  # Ensure this path matches your project structure
import numpy as np

# Load configuration
import yaml
with open("../configs/config.yaml", "r") as f:
    config = yaml.safe_load(f)

DATA_PATH = config["paths"]["data"]
BATCH_SIZE = config["hyperparameters"]["batch_size"]
NUM_WORKERS = config["data"]["num_workers"]

# Initialize dataset
print("Initializing dataset...")
dataset = ProjectDatasets(root_path=DATA_PATH)

# Split dataset (assuming a simple split for demonstration purposes)
data_length = len(dataset)
train_length = int(data_length * config["dataset_split"]["train"])
val_length = int(data_length * config["dataset_split"]["val"])
test_length = data_length - train_length - val_length

data_splits = torch.utils.data.random_split(dataset, [train_length, val_length, test_length])
train_dataset, val_dataset, test_dataset = data_splits

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=config["data"]["shuffle"], num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Display Dataset Information
print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples, {len(test_dataset)} test samples")

# Analyze Dataset
class_names = config["class_names"]
num_classes = len(class_names)

# Initialize counters
pixel_counts = np.zeros(num_classes, dtype=np.int64)
class_presence = np.zeros(num_classes, dtype=np.int64)

def analyze_dataset(loader):
    global pixel_counts, class_presence
    for batch in loader:
        _, masks, _, _ = batch
        masks = masks.numpy()  # Convert to numpy for easier processing

        # Count pixels per class
        for class_idx in range(num_classes):
            pixel_counts[class_idx] += np.sum(masks == class_idx)

        # Check presence of classes in each mask
        unique_classes = np.unique(masks)
        for cls in unique_classes:
            class_presence[cls] += 1

print("Analyzing training dataset...")
analyze_dataset(train_loader)
print("Analyzing validation dataset...")
analyze_dataset(val_loader)
print("Analyzing test dataset...")
analyze_dataset(test_loader)

# Total pixel count
total_pixels = np.sum(pixel_counts)

# Class distribution (percentage of total pixels)
class_distribution = (pixel_counts / total_pixels) * 100

# Print summary
print("\nDataset Analysis Summary:")
print(f"Total images: {data_length}")
print(f"Classes present: {np.nonzero(class_presence)[0].tolist()}")
print("Pixel count per class:")
for idx, count in enumerate(pixel_counts):
    print(f"  {class_names[idx]}: {count} pixels ({class_distribution[idx]:.2f}%)")

print("\nClass presence across images:")
for idx, presence in enumerate(class_presence):
    print(f"  {class_names[idx]}: Present in {presence} images")

Initializing dataset...
Dataset split: 5936 training samples, 742 validation samples, 743 test samples
Analyzing training dataset...


Analyzing validation dataset...
Analyzing test dataset...

Dataset Analysis Summary:
Total images: 7421
Classes present: [0, 7, 8, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
Pixel count per class:
  Unlabeled: 45673310 pixels (9.39%)
  Unused: 0 pixels (0.00%)
  Unused: 0 pixels (0.00%)
  Unused: 0 pixels (0.00%)
  Static: 0 pixels (0.00%)
  Dynamic: 0 pixels (0.00%)
  Ground: 0 pixels (0.00%)
  Road: 120385177 pixels (24.75%)
  Sidewalk: 7639300 pixels (1.57%)
  Parking: 0 pixels (0.00%)
  Rail track: 0 pixels (0.00%)
  Building: 87507849 pixels (17.99%)
  Wall: 3042954 pixels (0.63%)
  Fence: 5054439 pixels (1.04%)
  Guard rail: 1 pixels (0.00%)
  Bridge: 1 pixels (0.00%)
  Tunnel: 0 pixels (0.00%)
  Pole: 0 pixels (0.00%)
  Pole group: 1737313 pixels (0.36%)
  Traffic light: 383637 pixels (0.08%)
  Traffic sign: 1901335 pixels (0.39%)
  Vegetation: 76484860 pixels (15.73%)
  Terrain: 6979597 pixels (1.44%)
  Sky: 103826098 pixels (21.35%)
  P

In [8]:
import os
import torch
from torch.utils.data import DataLoader
from src.dataset import ProjectDatasets  # Ensure this path matches your project structure
import numpy as np

# Load configuration
import yaml
with open("../configs/config.yaml", "r") as f:
    config = yaml.safe_load(f)

DATA_PATH = config["paths"]["data"]
BATCH_SIZE = config["hyperparameters"]["batch_size"]
NUM_WORKERS = config["data"]["num_workers"]
SEED = config["hyperparameters"]["seed"]

# Set random seed for reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)

# Initialize dataset
print("Initializing dataset...")
dataset = ProjectDatasets(root_path=DATA_PATH)

# Split dataset
data_length = len(dataset)
train_length = int(data_length * config["dataset_split"]["train"])
val_length = int(data_length * config["dataset_split"]["val"])
test_length = data_length - train_length - val_length

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_length, val_length, test_length]
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=config["data"]["shuffle"], num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Display Dataset Information
print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples, {len(test_dataset)} test samples")

# Analyze Dataset
class_names = config["class_names"]
num_classes = len(class_names)

# Initialize counters
statistics = {
    "train": {"sunny": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)},
               "rainy": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)}},
    "val": {"sunny": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)},
             "rainy": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)}},
    "test": {"sunny": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)},
              "rainy": {"pixel_counts": np.zeros(num_classes, dtype=np.int64), "class_presence": np.zeros(num_classes, dtype=np.int64)}}
}

def analyze_dataset(loader, dataset_type):
    for batch in loader:
        _, masks, sources, _ = batch
        masks = masks.numpy()  # Convert to numpy for easier processing

        for i, src in enumerate(sources):
            src = src.lower()
            mask = masks[i]

            # Update stats for sunny or rainy
            if src in statistics[dataset_type]:
                for class_idx in range(num_classes):
                    statistics[dataset_type][src]["pixel_counts"][class_idx] += np.sum(mask == class_idx)

                unique_classes = np.unique(mask)
                for cls in unique_classes:
                    statistics[dataset_type][src]["class_presence"][cls] += 1

print("Analyzing training dataset...")
analyze_dataset(train_loader, "train")
print("Analyzing validation dataset...")
analyze_dataset(val_loader, "val")
print("Analyzing test dataset...")
analyze_dataset(test_loader, "test")

# Summarize statistics
def summarize_statistics(stats, category):
    print(f"\n{category.capitalize()} Dataset Analysis Summary:")

    for condition in ["sunny", "rainy"]:
        print(f"\n  Condition: {condition.capitalize()}")
        total_pixels = np.sum(stats[condition]["pixel_counts"])
        class_distribution = (stats[condition]["pixel_counts"] / total_pixels) * 100 if total_pixels > 0 else np.zeros(num_classes)

        print("  Pixel count per class:")
        for idx, count in enumerate(stats[condition]["pixel_counts"]):
            print(f"    {class_names[idx]}: {count} pixels ({class_distribution[idx]:.2f}%)")

        print("  Class presence across images:")
        for idx, presence in enumerate(stats[condition]["class_presence"]):
            print(f"    {class_names[idx]}: Present in {presence} images")

print("\nTraining Dataset Analysis:")
summarize_statistics(statistics["train"], "training")
print("\nValidation Dataset Analysis:")
summarize_statistics(statistics["val"], "validation")
print("\nTest Dataset Analysis:")
summarize_statistics(statistics["test"], "test")


Initializing dataset...
Dataset split: 5936 training samples, 742 validation samples, 743 test samples
Analyzing training dataset...


Analyzing validation dataset...
Analyzing test dataset...

Training Dataset Analysis:

Training Dataset Analysis Summary:

  Condition: Sunny
  Pixel count per class:
    Unlabeled: 16241374 pixels (8.24%)
    Unused: 0 pixels (0.00%)
    Unused: 0 pixels (0.00%)
    Unused: 0 pixels (0.00%)
    Static: 0 pixels (0.00%)
    Dynamic: 0 pixels (0.00%)
    Ground: 0 pixels (0.00%)
    Road: 49933240 pixels (25.33%)
    Sidewalk: 3872136 pixels (1.96%)
    Parking: 0 pixels (0.00%)
    Rail track: 0 pixels (0.00%)
    Building: 6945105 pixels (3.52%)
    Wall: 1241694 pixels (0.63%)
    Fence: 2143578 pixels (1.09%)
    Guard rail: 0 pixels (0.00%)
    Bridge: 1 pixels (0.00%)
    Tunnel: 0 pixels (0.00%)
    Pole: 0 pixels (0.00%)
    Pole group: 414028 pixels (0.21%)
    Traffic light: 200825 pixels (0.10%)
    Traffic sign: 347781 pixels (0.18%)
    Vegetation: 40889517 pixels (20.74%)
    Terrain: 4682812 pixels (2.38%)
    Sky: 64138014 pixels (32.54%)
    Person: 261897 pixels (0.13%

In [9]:
# Summarize statistics into a table
import pandas as pd
def create_statistics_table(stats):
    table_data = []

    for class_idx, class_name in enumerate(class_names):
        row = {"Class": class_name, "Total Pixel Count": 0, "Total Class Presence": 0,
               "Train Sunny Pixel Count": stats["train"]["sunny"]["pixel_counts"][class_idx],
               "Train Sunny Class Presence": stats["train"]["sunny"]["class_presence"][class_idx],
               "Train Rainy Pixel Count": stats["train"]["rainy"]["pixel_counts"][class_idx],
               "Train Rainy Class Presence": stats["train"]["rainy"]["class_presence"][class_idx],
               "Val Sunny Pixel Count": stats["val"]["sunny"]["pixel_counts"][class_idx],
               "Val Sunny Class Presence": stats["val"]["sunny"]["class_presence"][class_idx],
               "Val Rainy Pixel Count": stats["val"]["rainy"]["pixel_counts"][class_idx],
               "Val Rainy Class Presence": stats["val"]["rainy"]["class_presence"][class_idx],
               "Test Sunny Pixel Count": stats["test"]["sunny"]["pixel_counts"][class_idx],
               "Test Sunny Class Presence": stats["test"]["sunny"]["class_presence"][class_idx],
               "Test Rainy Pixel Count": stats["test"]["rainy"]["pixel_counts"][class_idx],
               "Test Rainy Class Presence": stats["test"]["rainy"]["class_presence"][class_idx]}

        row["Total Pixel Count"] = row["Train Sunny Pixel Count"] + row["Train Rainy Pixel Count"] + row["Val Sunny Pixel Count"] + row["Val Rainy Pixel Count"] + row["Test Sunny Pixel Count"] + row["Test Rainy Pixel Count"]
        row["Total Class Presence"] = row["Train Sunny Class Presence"] + row["Train Rainy Class Presence"] + row["Val Sunny Class Presence"] + row["Val Rainy Class Presence"] + row["Test Sunny Class Presence"] + row["Test Rainy Class Presence"]

        table_data.append(row)

    return pd.DataFrame(table_data)


# Generate and display the table
statistics_table = create_statistics_table(statistics)

# Display the table in the notebook
from IPython.display import display
display(statistics_table)

,Class,Total Pixel Count,Total Class Presence,Train Sunny Pixel Count,Train Sunny Class Presence,Train Rainy Pixel Count,Train Rainy Class Presence,Val Sunny Pixel Count,Val Sunny Class Presence,Val Rainy Pixel Count,Val Rainy Class Presence,Test Sunny Pixel Count,Test Sunny Class Presence,Test Rainy Pixel Count,Test Rainy Class Presence
0,Unlabeled,45673310,7421,16241374,3008,20350854,2928,2130826,394,2412829,348,2016293,377,2521134,366
1,Unused,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Unused,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Unused,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Static,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Dynamic,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Ground,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Road,120385177,7421,49933240,3008,46321954,2928,6548338,394,5488690,348,6241276,377,5851679,366
8,Sidewalk,7639300,6746,3872136,2526,2203493,2883,507139,332,266118,338,507269,311,283145,356
9,Parking,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# Create Image Count Table
def create_image_count_table(stats):
    table_data = []

    for class_idx, class_name in enumerate(class_names):
        row = {"Class": class_name,
               "Train Sunny Image Count": stats["train"]["sunny"]["class_presence"][class_idx],
               "Train Rainy Image Count": stats["train"]["rainy"]["class_presence"][class_idx],
               "Val Sunny Image Count": stats["val"]["sunny"]["class_presence"][class_idx],
               "Val Rainy Image Count": stats["val"]["rainy"]["class_presence"][class_idx],
               "Test Sunny Image Count": stats["test"]["sunny"]["class_presence"][class_idx],
               "Test Rainy Image Count": stats["test"]["rainy"]["class_presence"][class_idx]}

        row["Total Image Count"] = row["Train Sunny Image Count"] + row["Train Rainy Image Count"] + \
                                   row["Val Sunny Image Count"] + row["Val Rainy Image Count"] + \
                                   row["Test Sunny Image Count"] + row["Test Rainy Image Count"]

        table_data.append(row)

    return pd.DataFrame(table_data)

# Create Pixel Count Table
def calculate_class_weights(stats):
    """
    Calculate the class weights based on the pixel counts from the dataset.
    The weight is calculated as:
        weight = total_pixels / class_pixels
    Classes with less than 20 pixels will have 0 weight.
    """
    total_pixels = sum([
        sum(stats["train"]["sunny"]["pixel_counts"]),
        sum(stats["train"]["rainy"]["pixel_counts"]),
        sum(stats["val"]["sunny"]["pixel_counts"]),
        sum(stats["val"]["rainy"]["pixel_counts"]),
        sum(stats["test"]["sunny"]["pixel_counts"]),
        sum(stats["test"]["rainy"]["pixel_counts"])
    ])
    
    class_weights = []

    for class_idx, class_name in enumerate(class_names):
        # Calculate total pixel count for the class across all datasets
        total_class_pixels = (
            stats["train"]["sunny"]["pixel_counts"][class_idx] +
            stats["train"]["rainy"]["pixel_counts"][class_idx] +
            stats["val"]["sunny"]["pixel_counts"][class_idx] +
            stats["val"]["rainy"]["pixel_counts"][class_idx] +
            stats["test"]["sunny"]["pixel_counts"][class_idx] +
            stats["test"]["rainy"]["pixel_counts"][class_idx]
        )
        
        if total_class_pixels < 20:
            class_weights.append(0)  # Assign 0 weight for classes with less than 20 pixels
        else:
            # Calculate the weight based on the formula: total_pixels / class_pixels
            weight = total_pixels / total_class_pixels
            class_weights.append(weight)

    # Normalize the weights by dividing each weight by the sum of all weights
    total_weight = sum(class_weights)
    normalized_weights = [weight / total_weight if weight > 0 else 0 for weight in class_weights]

    return class_weights, normalized_weights

def create_pixel_count_table_with_weights(stats):
    table_data = []
    class_weights, normalized_weights = calculate_class_weights(stats)

    for class_idx, class_name in enumerate(class_names):
        row = {
            "Class": class_name,
            "Train Sunny Pixel Count": stats["train"]["sunny"]["pixel_counts"][class_idx],
            "Train Rainy Pixel Count": stats["train"]["rainy"]["pixel_counts"][class_idx],
            "Val Sunny Pixel Count": stats["val"]["sunny"]["pixel_counts"][class_idx],
            "Val Rainy Pixel Count": stats["val"]["rainy"]["pixel_counts"][class_idx],
            "Test Sunny Pixel Count": stats["test"]["sunny"]["pixel_counts"][class_idx],
            "Test Rainy Pixel Count": stats["test"]["rainy"]["pixel_counts"][class_idx]
        }

        row["Total Pixel Count"] = row["Train Sunny Pixel Count"] + row["Train Rainy Pixel Count"] + \
                                   row["Val Sunny Pixel Count"] + row["Val Rainy Pixel Count"] + \
                                   row["Test Sunny Pixel Count"] + row["Test Rainy Pixel Count"]
        
        # Assign weight and normalized weight
        row["Weight"] = class_weights[class_idx]
        row["Normalized Weight"] = normalized_weights[class_idx]

        table_data.append(row)

    return pd.DataFrame(table_data)

# Generate and display the tables
image_count_table = create_image_count_table(statistics)
pixel_count_table = create_pixel_count_table_with_weights(statistics)

# # Display the table in the notebook
# from IPython.display import display
# tools.display_dataframe_to_user(name="Image Count Table", dataframe=image_count_table)
# tools.display_dataframe_to_user(name="Pixel Count Table", dataframe=pixel_count_table)


In [14]:
display(create_image_count_table(statistics))

,Class,Train Sunny Image Count,Train Rainy Image Count,Val Sunny Image Count,Val Rainy Image Count,Test Sunny Image Count,Test Rainy Image Count,Total Image Count
0,Unlabeled,3008,2928,394,348,377,366,7421
1,Unused,0,0,0,0,0,0,0
2,Unused,0,0,0,0,0,0,0
3,Unused,0,0,0,0,0,0,0
4,Static,0,0,0,0,0,0,0
5,Dynamic,0,0,0,0,0,0,0
6,Ground,0,0,0,0,0,0,0
7,Road,3008,2928,394,348,377,366,7421
8,Sidewalk,2526,2883,332,338,311,356,6746
9,Parking,0,0,0,0,0,0,0


In [15]:
display(create_pixel_count_table_with_weights(statistics))

,Class,Train Sunny Pixel Count,Train Rainy Pixel Count,Val Sunny Pixel Count,Val Rainy Pixel Count,Test Sunny Pixel Count,Test Rainy Pixel Count,Total Pixel Count
0,Unlabeled,16241374,20350854,2130826,2412829,2016293,2521134,45673310
1,Unused,0,0,0,0,0,0,0
2,Unused,0,0,0,0,0,0,0
3,Unused,0,0,0,0,0,0,0
4,Static,0,0,0,0,0,0,0
5,Dynamic,0,0,0,0,0,0,0
6,Ground,0,0,0,0,0,0,0
7,Road,49933240,46321954,6548338,5488690,6241276,5851679,120385177
8,Sidewalk,3872136,2203493,507139,266118,507269,283145,7639300
9,Parking,0,0,0,0,0,0,0
